In [6]:
Data from Chicago Open Data Portal

Maps from Mapbox using Mapboxgl-Jupyter Python Visualization library



The following command must be run outside of the IPython shell:

    $ pip install mapboxgl

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [14]:
pwd

'/home/vagrant'

In [7]:
from mapboxgl.viz import *
from mapboxgl.utils import *
import pandas as pd

In [3]:
import pandas as pd
# Get the latest Chicago Open Data on gun violence (2001-present)
# https://data.cityofchicago.org/Public-Safety/Gun-Crimes-Heat-Map/iinq-m3rg/data
df = pd.read_csv('https://data.cityofchicago.org/api/views/iinq-m3rg/rows.csv?accessType=DOWNLOAD',
                parse_dates=['Date', 'Updated On'], infer_datetime_format=True)

In [4]:
print(df.shape)
df.head(1)

(61463, 22)


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10000092,HY189866,2015-03-18 19:44:00,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,...,28,25,04B,1144606.0,1903566.0,2015,2018-02-10 15:50:01,41.891399,-87.744385,"(41.891398861, -87.744384567)"


In [8]:
# only keep incidents that have a location
df.dropna(subset=['Latitude', 'Longitude'], inplace=True, axis=0, how='any')
df.shape

# All 51k incidnets
df_to_geojson(df, filename='all.geojson',
              lat='Latitude', lon='Longitude',
             properties=['Primary Type', 'Description'])
{'feature_count': 51883, 'filename': 'all.geojson', 'type': 'file'}
# 2017 only data, 15k incidents
df_to_geojson(df[df['Year']==2017], filename='lastyear.geojson',
              lat='Latitude', lon='Longitude',
             properties=['Primary Type', 'Description'])
{'feature_count': 14666, 'filename': 'lastyear.geojson', 'type': 'file'}

{'feature_count': 14666, 'filename': 'lastyear.geojson', 'type': 'file'}

In [11]:
# Set Mapbox Access Token to create visual.  https://www.mapbox.com/account/access-tokens
token = 'pk.eyJ1IjoiZGF2aWRwb2ZvIiwiYSI6ImNqbGZzYzk3MTEwMWUza3BobzNra25ldmkifQ.w5mtODwrb_MAFZrimaAkXw'
#Create a heatmap style
heatmap_color_stops = create_color_stops([0.1,0.25,0.5,0.75,1], colors='YlOrRd')
heatmap_radius_stops = [[9, 15], [20, 100]] #increase radius with zoom
intensity_stops = [[9, 0.01], [20, 0.3]]

#Create a heatmap 
heatmap = HeatmapViz('all.geojson', 
                  access_token=token,
                  color_stops = heatmap_color_stops,
                  radius_stops = heatmap_radius_stops,
                  intensity_stops = intensity_stops,
                  opacity = 0.9,
                  center = (-87.74, 41.85),
                  zoom = 9.5,
                  below_layer='waterway-label'
                 )
# Density of incidents
heatmap.show()


In [13]:


# Create a circle map of Chicago gun incidents by Ward
from mapboxgl.colors import color_ramps

# Get a categorical color ramp from the mapboxgl color brewer defaults
color_ramp = color_ramps['Set1'][5]

# Create a categorical color ramp for each unique `Ward` in the map 
category_color_stops = []
for i, type in enumerate(df['Primary Type'].unique().tolist()): 
    category_color_stops.append([type, color_ramp[i]])

#Create a circle map of each incident colored by Ward
wardmap = CircleViz('all.geojson', 
                  access_token=token,
                  opacity = 0.9,
                  height = '600px',
                  center = (-87.74, 41.85),
                  zoom = 9.5,
                  below_layer='waterway-label'
                 )

wardmap.color_property = 'Primary Type'
wardmap.color_function_type = 'match'
wardmap.color_stops = category_color_stops
wardmap.stroke_color = 'white'
wardmap.stroke_width = 0.1
# Type of gun violence incidents
wardmap.show()

#What's the difference to 2017 gun incidents in Chicago?
heatmap.data = 'lastyear.geojson'
heatmap.show()

wardmap.data = 'lastyear.geojson'
wardmap.show()